In [1]:
### Almost all code in this cell by https://github.com/Vsevolod-pl

import cv2
import json
import argparse
import treepoem
import numpy as np
from matplotlib import pyplot as plt
from typing import Optional


def load_json(fname, *args, **kwargs):
    with open(fname) as f:
        return json.load(f, *args, **kwargs)


def save_json(jd, fname, *args, indent=4, **kwargs):
    with open(fname, 'w') as f:
        json.dump(jd, f, *args, indent=indent, **kwargs)


def aligned_affine(bar, M, fix_position=True):
    xps = [0, 0, 1, 1]
    yps = [1, 0, 0, 1]
    
    height, width, _ = bar.shape
    corners = np.array([[x*width, y*height, 1] for x, y in zip(xps, yps)]).T
    
    if fix_position:
        M = M.copy()
        M[:,-1] *= 0
        M[:,-1] = -np.min(M@corners, axis=-1)

    new_sz = np.ceil(np.max(M@corners, axis=-1)).astype(np.int32)
    img = cv2.warpAffine(bar, M, new_sz)
    return img, M@corners


def generate_perspective_distort(img, alpha=0.1, beta=0.01):
    xps = [0, 0, 1, 1]
    yps = [1, 0, 0, 1]
    height, width, _ = img.shape
    corners = np.array([[x*width, y*height, 1] for x, y in zip(xps, yps)]).T

    M = np.zeros((3,3))
    M[:-1,:-1] = np.random.randn(2, 2)*alpha + np.eye(2)*(1.-alpha)
    M[-1, :-1] = beta*np.abs(np.random.randn(1,2))
    M[:-1,-1] *= 0
    M[-1,-1] = 1
    coords = (M@corners)[:-1]
    M[:-1,-1] = -np.min(coords, axis=-1)
    return M

def generate_aligned_perspective_distort(img, distribution, **distribution_params):
    xps = [0, 0, 1, 1]
    yps = [1, 0, 0, 1]
    height, width, _ = img.shape
    corners = np.array([[x*width, y*height] for x, y in zip(xps, yps)])
    corners_old = corners.copy()

    dx, dy = distribution(**distribution_params, size=corners.shape).T # np.random.exponential(scale=width*scale, size=corners.shape).T

    corners[0,0] -= dx[0]
    corners[0,1] += dy[0]

    corners[1,0] -= dx[1]
    corners[1,1] -= dy[1]

    corners[2,0] += dx[2]
    corners[2,1] -= dy[2]

    corners[3,0] += dx[3]
    corners[3,1] += dy[3]

    theta = np.math.pi*np.random.random()*2
    c, s = np.cos(theta), np.sin(theta)
    R = np.array(((c, -s), (s, c)))

    corners = corners@R

    M = cv2.getPerspectiveTransform(corners_old.astype(np.float32), corners.astype(np.float32))

    corners = np.array([[x*width, y*height, 1] for x, y in zip(xps, yps)]).T
    M[:-1,-1] *= 0
    M[-1,-1] = 1
    coords = (M@corners)[:-1]
    M[:-1,-1] = -np.min(coords, axis=-1)
    coords = M@corners
    coords = coords[:-1]/coords[-1]

    new_sz = np.ceil(np.max(coords, axis=-1)).astype(np.int32)

    return M


def aligned_perspective(img, M):
    xps = [0, 0, 1, 1]
    yps = [1, 0, 0, 1]
    height, width, _ = img.shape
    corners = np.array([[x*width, y*height, 1] for x, y in zip(xps, yps)]).T
    coords = M@corners
    coords = coords[:-1]/coords[-1]
    
    new_sz = np.ceil(np.max(coords, axis=-1)).astype(np.int32)
    
    img = cv2.warpPerspective(img, M, new_sz)
    return img, coords


def coords_to_regions(coords, dimensions):
    res = []
    for i in range(len(coords)):
        ptsx, ptsy = coords[i]
        
        res.append({\
        'shape_attributes': {
            'name': 'polygon',
            'all_points_x': list(ptsx),
            'all_points_y': list(ptsy)
        },
        'region_attributes': {'barcode': dimensions[i]}})
    return res


def export(img, name, coords, dimensions):
    #np.clip(img, 0, 1)
    plt.imsave(f'{name}.jpg', img)
    res = {f'{name}.jpg813086': {'filename': f'../code/{name}.jpg',
    'size': 813086,
    'regions': coords_to_regions(coords, dimensions),
    'file_attributes': {}}}
    save_json(res, f'{name}.json')


def generate_distorted(barcode_types, content_barcodes, source_img=None, distortions=None, opt_params: Optional[dict] = None):
    assert isinstance(opt_params, dict), 'Empty optmization space'
    barimgs = [treepoem.generate_barcode(typ, content) for typ, content in zip(barcode_types, content_barcodes)]
    if distortions is None:
        distribution = opt_params['distribution']
        distribution_params = opt_params['distribution_params']
        distortions = [generate_aligned_perspective_distort(np.array(img), distribution, **distribution_params) for img in barimgs]
    imgs, coords = zip(*[aligned_perspective(np.array(img), dis) for img, dis in zip(barimgs, distortions)])
    masks, _ = zip(*[aligned_perspective(np.ones_like(img), dis) for img, dis in zip(barimgs, distortions)])
    
    if source_img is None:
        width, height, _ = np.max([img.shape for img in imgs], axis=0)*len(imgs)//3
        combined = np.full((width, height, 3), fill_value=np.mean([np.mean(b) for b in barimags]), dtype=imgs[0].dtype)
    else:
        combined = plt.imread(source_img)[:,:,:3]
        width, height, _ = combined.shape

    for i in range(len(imgs)):
        w, h, _ = imgs[i].shape
        if width - w < 0 or height -h < 0:
            continue
        dw = np.random.randint(0, width - w)
        dh = np.random.randint(0, height - h)
        coords[i][0] += dh
        coords[i][1] += dw
        expanded_img = np.zeros_like(combined)
        expanded_img[dw:w+dw, dh:h+dh] = imgs[i]
        expanded_mask = np.zeros_like(combined)
        expanded_mask[dw:w+dw, dh:h+dh] = masks[i]
        combined = combined*(1-expanded_mask) + expanded_img
    return combined, coords, barimgs


In [2]:
import optuna
from itertools import combinations

def shrinked_gauss(size, mu: Optional[float] = None, sigma: Optional[float] = None):
    assert mu is not None and sigma is not None, 'Some params were not passed to function'
    return np.abs(np.random.randn(*size))

/homes/lbelkov/miniconda3/envs/hubenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def compute_angle(coords):
    results = []
    for c in coords:
        p = c.T[0]
        l2 = np.sqrt(np.power(p - c.T[1:], 2))
        indices = np.argsort(l2)
        vecs = p - c.T[1:][indices][:-1]
        results.append(np.arccos(np.dot(vecs[0], vecs[1]) / np.prod(np.linalg.norm(vecs, axis=1, ord=2))) * 180 / np.pi)
    return results

In [4]:
def objective(trial):
    params = dict(
        distribution = trial.suggest_categorical('distribution', [np.random.exponential, shrinked_gauss])
    )
    if params['distribution'] == np.random.exponential:
        params['distribution_params'] = dict(
            scale = trial.suggest_float('scale', 0.1, 1.0)
        )
    elif params['distribution'] == shrinked_gauss:
        params['distribution_params'] = dict(
            mu = trial.suggest_float('mu', 0.0, 1.0),
            sigma = trial.suggest_float('sigma', 0.1, 2.0)
        )
    
    conf = load_json('test_conf.json')
    results = []
    for _ in range(8):
        results.append(generate_distorted(conf['barcode_types'], conf['barcode_contents'], conf['source_img'], opt_params=params))

    angles = []
    for r in results:
        angles.extend(compute_angle(r[1]))

    min_angle = np.min(angles)
    min_angle = min_angle if min_angle <= 15 else 90.0

    # variance = np.mean(sum([np.power((r[0] - r[0].min())/(r[0].max() - r[0].min()), 2) for r in results])) - np.mean(sum([(r[0] - r[0].min())/(r[0].max() - r[0].min()) for r in results])**2)

    pairwise_loss = []
    for first, second in combinations(results, 2):
        f = first[0]
        f = (f - f.min()) / (f.max() - f.min())
        s = second[0]
        s = (s - s.min()) / (s.max() - s.min())
        pairwise_loss.append(np.power(f - s, 2).mean())

    return np.mean(pairwise_loss), min_angle

In [5]:
import warnings

In [6]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    study = optuna.create_study(directions=['maximize', 'maximize'])
    study.optimize(objective, n_trials=128, n_jobs=8, gc_after_trial=True, show_progress_bar=True)

[I 2024-05-15 10:26:54,866] A new study created in memory with name: no-name-353a7715-ef22-469c-a803-07b4e889926f
  1%|          | 1/128 [00:12<26:43, 12.62s/it]

[I 2024-05-15 10:27:07,450] Trial 4 finished with values: [0.133389683870825, 90.0] and parameters: {'distribution': <function shrinked_gauss at 0x7f96f4319080>, 'mu': 0.8516561761455654, 'sigma': 1.9173251073654651}. 


  2%|▏         | 2/128 [00:12<11:18,  5.38s/it]

[I 2024-05-15 10:27:07,759] Trial 5 finished with values: [0.14753701208479036, 90.0] and parameters: {'distribution': <function shrinked_gauss at 0x7f96f4319080>, 'mu': 0.5597805691695986, 'sigma': 1.6721363646123544}. 
[I 2024-05-15 10:27:07,827] Trial 3 finished with values: [0.14970587500940827, 90.0] and parameters: {'distribution': <built-in method exponential of numpy.random.mtrand.RandomState object at 0x7f97440abb40>, 'scale': 0.9387601513222117}. 


  4%|▍         | 5/128 [00:13<04:17,  2.09s/it]

...

 98%|█████████▊| 125/128 [03:29<00:03,  1.09s/it]

[I 2024-05-15 10:30:23,984] Trial 124 finished with values: [0.15121802166799905, 90.0] and parameters: {'distribution': <function shrinked_gauss at 0x7f96f4319080>, 'mu': 0.6959871009245481, 'sigma': 1.560239568777599}. 


 99%|█████████▉| 127/128 [03:29<00:00,  1.34it/s]

[I 2024-05-15 10:30:24,579] Trial 126 finished with values: [0.14441498994078386, 90.0] and parameters: {'distribution': <built-in method exponential of numpy.random.mtrand.RandomState object at 0x7f97440abb40>, 'scale': 0.13733739164174535}. 
[I 2024-05-15 10:30:24,753] Trial 127 finished with values: [0.15237944683210974, 90.0] and parameters: {'distribution': <function shrinked_gauss at 0x7f96f4319080>, 'mu': 0.2176713310014191, 'sigma': 0.16269447590900676}. 


100%|██████████| 128/128 [03:29<00:00,  1.64s/it]


In [7]:
best_study = study.trials[np.argmax(list(map(lambda t: t.values[0], filter(lambda t: t.values[1] > 15, study.trials))))]


In [ ]:
best_study

### Pairwise simillarity

 values=[0.16011994043042055, 90.0]

{'distribution': <function RandomState.exponential>,
 'scale': 0.7673727163364142}